## Coadd Metric Viewer

Displays metrics associated with deepCoadds as a series of tables. Metrics that deviate from the column mean by more than 3 standard deviations are highlighted in red.

Bear in mind highlighting assumes a gaussian-ish distribution, meaning that some outlier values may not be highlighted (and vice versa).

Columns named after a band show the number of exposures of that band that have gone into making the coadd (so, "200" under "g" for tract row "500" means that that 200 g-band exposures have gone into creating all the coadds that cover tract 500). 

In [9]:
day_obs = '20250604'
repo = '/repo/main'
collection = 'LSSTCam/runs/DRP/20250604_20250814/w_2025_33/DM-52202'
instrument = 'LSSTCam'

In [10]:
import lsst.daf.butler as dafButler

import operator
import numpy as np
import pandas as pd
from scipy.stats import zscore

from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-child {overflow: scroll}</style>"))

In [11]:
instrUrl = instrument
collectionUrl = 'runs/DRP/20250604_20250814/w_2025_33/DM-52202'
plotNavUrl = 'https://usdf-rsp-dev.slac.stanford.edu/plot-navigator'

In [12]:
urlDict = {'g_highSNStars_median':'', 'g_highSNStars_sigmaMad':'', 'g_highSNStars_count':''}
threshDict = {'numBlends':50, 'meanBlendIterations':25, 'meanBlendLogL':-1e5}
threshTypeDict = {'numBlends':'lt', 'meanBlendIterations':'ge', 'meanBlendLogL':'lt'}

In [13]:
def metricBundletToDataFrame(bundleName, refs):
    data = []
    for ref in refs:
        metrics = butler.get(ref)
        row = {'Tract':ref.dataId['tract']}
        for metric in metrics[bundleName]:
            key = '{} {}'.format(metric.metric_name.metric, metric.quantity.unit.to_string())
            value = metric.quantity.value
            row[key] = value
        data.append(row)
    
    df = pd.DataFrame(data)
    df.set_index('Tract', inplace=True)
    return df

def build_urls(s, names=None):
    # Build URLs if column has an associated Plot Navigator page.
    colName = s.name.split()[0]
    if colName not in urlDict:
        return np.array(['-' if np.isnan(v) else f'{v:.2f}' for v in s.values])
    else:
        urlVal = urlDict[colName]
        return [f'<a href="{plotNavUrl}/tract/{index}/{instrUrl}/{collectionUrl}/{urlDict[colName]}">{val:.2f}</a>' for index, val in zip(s.index.values, s.values)]

def flagCells(s):
    colName = s.name.split()[0]
    if colName not in threshDict:
        return np.array(["" for x in s])
    else:
        mask = getattr(operator, threshTypeDict[colName])(s.values, threshDict[colName])
        return np.where(mask, "background-color: red","")

def displayTable(tractTable, metricTable):
    merged = pd.merge(tractTable, metricTable, on='Tract')
    style_df = merged.apply(flagCells, axis=0)
    urls = merged.apply(build_urls, axis=0)
    flagged = urls.style.apply(lambda x: style_df, axis=None)
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 50):
        display(flagged)

In [14]:
butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

In [16]:
visits = list(registry.queryDimensionRecords('visit', where=f"day_obs in ({day_obs}) AND instrument='{instrument}'"))
visitIds = [visit.id for visit in visits]
minId, maxId = np.min(visitIds), np.max(visitIds)

coaddRefs = list(registry.queryDatasets('deep_coadd', where=f"visit in ({minId}..{maxId}) AND instrument='{instrument}'"))
data = []
for coaddRef in coaddRefs:
    data.append({'Tract':coaddRef.dataId['tract'], 'Band':coaddRef.dataId['band']})
tractTable = pd.DataFrame(data)
tractTable = tractTable.groupby(tractTable.columns.tolist(),as_index=False).size()
tractTable = tractTable.pivot(index='Tract', columns='Band', values='size')

## Object Table Core Metrics:

In [19]:
refs = list(registry.queryDatasets('object_metrics'))

In [21]:
### Blend Metrics

In [22]:
#BlendLogL: Log likelihood of the entire blend in scarlet_lite.
#bundleName = 'blendMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### E1Diff Metrics

In [23]:
# Ellipticity Residuals (e1 - e1_psf) per tract per band
# highSNStars are SNR > 2700, lowSNStars are SNR > 500
# _count is number of stars in each SNR bin. 
bundleName = 'e1Diff'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,u_highSNStars_median pix,u_highSNStars_sigmaMad pix,u_highSNStars_count ct,u_lowSNStars_median pix,u_lowSNStars_sigmaMad pix,u_lowSNStars_count ct,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct,z_highSNStars_median pix,z_highSNStars_sigmaMad pix,z_highSNStars_count ct,z_lowSNStars_median pix,z_lowSNStars_sigmaMad pix,z_lowSNStars_count ct,y_highSNStars_median pix,y_highSNStars_sigmaMad pix,y_highSNStars_count ct,y_lowSNStars_median pix,y_lowSNStars_sigmaMad pix,y_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6341,25.00,88.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,178.00,0.00,0.00,9008.00,-0.00,0.00,1186.00,-0.00,0.00,13428.00,-0.00,0.00,1159.00,-0.00,0.00,12800.00,0.00,0.00,1076.00,-0.00,0.00,9080.00
6342,162.00,297.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,453.00,0.00,0.00,9180.00,-0.00,0.00,1419.00,-0.00,0.00,13179.00,0.00,0.00,1047.00,0.00,0.00,11853.00,0.00,0.00,1194.00,0.00,0.00,8751.00
6343,224.00,253.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,368.00,0.00,0.00,8265.00,0.00,0.00,1063.00,0.00,0.00,11813.00,0.00,0.00,737.00,0.00,0.00,10422.00,-0.00,0.00,697.00,0.00,0.00,7240.00
6344,227.00,377.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,419.00,0.00,0.00,7743.00,-0.00,0.00,825.00,-0.00,0.00,10422.00,-0.00,0.00,723.00,-0.00,0.00,10027.00,0.00,0.00,455.00,-0.00,0.00,6322.00
6345,227.00,187.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,370.00,-0.00,0.00,7141.00,-0.00,0.00,569.00,-0.00,0.00,9442.00,0.00,0.00,593.00,-0.00,0.00,8881.00,-0.00,0.00,357.00,-0.00,0.00,5659.00
6346,54.00,4.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,248.00,0.00,0.00,5878.00,-0.00,0.00,293.00,-0.00,0.00,7578.00,-0.00,0.00,122.00,-0.00,0.00,5945.00,0.00,0.01,246.00,0.00,0.00,4169.00
6571,29.00,33.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,413.00,0.00,0.00,10471.00,-0.00,0.00,998.00,-0.00,0.00,13287.00,0.00,0.00,1143.00,0.00,0.00,12512.00,0.00,0.00,999.00,0.00,0.00,9104.00
6572,228.00,342.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,445.00,0.00,0.00,9548.00,-0.00,0.00,1667.00,-0.00,0.00,13906.00,0.00,0.00,1734.00,0.00,0.00,12971.00,0.00,0.00,649.00,-0.00,0.00,9472.00
6573,746.00,387.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,420.00,0.00,0.00,9328.00,-0.00,0.00,2489.00,-0.00,0.00,15282.00,0.00,0.00,1673.00,0.00,0.00,12447.00,-0.00,0.00,903.00,-0.00,0.00,10166.00


### E2 Diff Metrics

In [24]:
bundleName = 'e2Diff'
etricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,u_highSNStars_median pix,u_highSNStars_sigmaMad pix,u_highSNStars_count ct,u_lowSNStars_median pix,u_lowSNStars_sigmaMad pix,u_lowSNStars_count ct,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct,z_highSNStars_median pix,z_highSNStars_sigmaMad pix,z_highSNStars_count ct,z_lowSNStars_median pix,z_lowSNStars_sigmaMad pix,z_lowSNStars_count ct,y_highSNStars_median pix,y_highSNStars_sigmaMad pix,y_highSNStars_count ct,y_lowSNStars_median pix,y_lowSNStars_sigmaMad pix,y_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6341,25.00,88.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,178.00,0.00,0.00,9008.00,-0.00,0.00,1186.00,-0.00,0.00,13428.00,-0.00,0.00,1159.00,-0.00,0.00,12800.00,0.00,0.00,1076.00,-0.00,0.00,9080.00
6342,162.00,297.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,453.00,0.00,0.00,9180.00,-0.00,0.00,1419.00,-0.00,0.00,13179.00,0.00,0.00,1047.00,0.00,0.00,11853.00,0.00,0.00,1194.00,0.00,0.00,8751.00
6343,224.00,253.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,368.00,0.00,0.00,8265.00,0.00,0.00,1063.00,0.00,0.00,11813.00,0.00,0.00,737.00,0.00,0.00,10422.00,-0.00,0.00,697.00,0.00,0.00,7240.00
6344,227.00,377.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,419.00,0.00,0.00,7743.00,-0.00,0.00,825.00,-0.00,0.00,10422.00,-0.00,0.00,723.00,-0.00,0.00,10027.00,0.00,0.00,455.00,-0.00,0.00,6322.00
6345,227.00,187.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,370.00,-0.00,0.00,7141.00,-0.00,0.00,569.00,-0.00,0.00,9442.00,0.00,0.00,593.00,-0.00,0.00,8881.00,-0.00,0.00,357.00,-0.00,0.00,5659.00
6346,54.00,4.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,248.00,0.00,0.00,5878.00,-0.00,0.00,293.00,-0.00,0.00,7578.00,-0.00,0.00,122.00,-0.00,0.00,5945.00,0.00,0.01,246.00,0.00,0.00,4169.00
6571,29.00,33.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,413.00,0.00,0.00,10471.00,-0.00,0.00,998.00,-0.00,0.00,13287.00,0.00,0.00,1143.00,0.00,0.00,12512.00,0.00,0.00,999.00,0.00,0.00,9104.00
6572,228.00,342.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,445.00,0.00,0.00,9548.00,-0.00,0.00,1667.00,-0.00,0.00,13906.00,0.00,0.00,1734.00,0.00,0.00,12971.00,0.00,0.00,649.00,-0.00,0.00,9472.00
6573,746.00,387.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,0.00,0.00,420.00,0.00,0.00,9328.00,-0.00,0.00,2489.00,-0.00,0.00,15282.00,0.00,0.00,1673.00,0.00,0.00,12447.00,-0.00,0.00,903.00,-0.00,0.00,10166.00


In [25]:
### Isolated Deblender Metrics

In [26]:
#bundleName = 'isolatedDeblenderMetrics'
#etricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Parent Deblender Metrics

In [27]:
#bundleName = 'parentDeblenderMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### PSF CModel Scatter Metrics

In [28]:
#bundleName = 'psfCModelScatter'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Shape Size Fractional Diff Metrics

In [29]:
bundleName = 'shapeSizeFractionalDiff'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,u_highSNStars_median pix,u_highSNStars_sigmaMad pix,u_highSNStars_count ct,u_lowSNStars_median pix,u_lowSNStars_sigmaMad pix,u_lowSNStars_count ct,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct,z_highSNStars_median pix,z_highSNStars_sigmaMad pix,z_highSNStars_count ct,z_lowSNStars_median pix,z_lowSNStars_sigmaMad pix,z_lowSNStars_count ct,y_highSNStars_median pix,y_highSNStars_sigmaMad pix,y_highSNStars_count ct,y_lowSNStars_median pix,y_lowSNStars_sigmaMad pix,y_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6341,25.00,88.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,178.00,-0.00,0.00,9008.00,-0.00,0.00,1186.00,-0.00,0.00,13428.00,-0.00,0.00,1159.00,0.00,0.00,12800.00,-0.00,0.00,1076.00,0.00,0.00,9080.00
6342,162.00,297.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,453.00,0.00,0.00,9180.00,-0.00,0.00,1419.00,0.00,0.00,13179.00,-0.00,0.00,1047.00,0.00,0.00,11853.00,-0.00,0.00,1194.00,-0.00,0.00,8751.00
6343,224.00,253.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,368.00,-0.00,0.00,8265.00,-0.00,0.00,1063.00,-0.00,0.00,11813.00,-0.00,0.00,737.00,0.00,0.00,10422.00,-0.00,0.01,697.00,-0.00,0.01,7240.00
6344,227.00,377.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,419.00,-0.00,0.00,7743.00,-0.00,0.00,825.00,0.00,0.00,10422.00,-0.01,0.00,723.00,-0.00,0.00,10027.00,-0.01,0.01,455.00,-0.00,0.00,6322.00
6345,227.00,187.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,370.00,-0.00,0.00,7141.00,-0.00,0.00,569.00,-0.00,0.00,9442.00,-0.01,0.00,593.00,-0.00,0.00,8881.00,-0.00,0.01,357.00,0.00,0.00,5659.00
6346,54.00,4.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,248.00,-0.00,0.00,5878.00,-0.00,0.00,293.00,-0.00,0.00,7578.00,-0.01,0.00,122.00,-0.00,0.00,5945.00,-0.02,0.02,246.00,-0.00,0.00,4169.00
6571,29.00,33.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,413.00,0.00,0.00,10471.00,-0.00,0.00,998.00,0.00,0.00,13287.00,-0.00,0.01,1143.00,0.00,0.00,12512.00,-0.00,0.00,999.00,0.00,0.00,9104.00
6572,228.00,342.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,445.00,0.00,0.00,9548.00,-0.00,0.00,1667.00,0.00,0.00,13906.00,-0.00,0.00,1734.00,0.00,0.00,12971.00,-0.00,0.00,649.00,0.00,0.00,9472.00
6573,746.00,387.00,-,-,0.00,-,-,0.00,nan,nan,0.00,-,-,0.00,-0.00,0.00,420.00,-0.00,0.00,9328.00,-0.00,0.00,2489.00,0.00,0.00,15282.00,-0.00,0.00,1673.00,0.00,0.00,12447.00,-0.00,0.00,903.00,-0.00,0.00,10166.00


### Skipped Deblender Metrics

In [30]:
#bundleName = 'skippedDeblenderMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Sky Flux Statistic Metrics

In [31]:
bundleName = 'skyFluxStatisticMetric'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,u_medianSky nJy,u_meanSky nJy,u_stdevSky nJy,u_sigmaMADSky nJy,g_medianSky nJy,g_meanSky nJy,g_stdevSky nJy,g_sigmaMADSky nJy,r_medianSky nJy,r_meanSky nJy,r_stdevSky nJy,r_sigmaMADSky nJy,i_medianSky nJy,i_meanSky nJy,i_stdevSky nJy,i_sigmaMADSky nJy,z_medianSky nJy,z_meanSky nJy,z_stdevSky nJy,z_sigmaMADSky nJy,y_medianSky nJy,y_meanSky nJy,y_stdevSky nJy,y_sigmaMADSky nJy
Tract,,,,,,,,,,,,,,,,,,,,,,,,,,
6341,25.00,88.00,-,-,-,-,-,-,-,-,-17.12,-24.37,366.19,224.26,-6.10,-8.21,366.78,224.15,-15.25,-19.45,476.34,338.02,-72.77,-88.49,1365.21,861.04
6342,162.00,297.00,-,-,-,-,-,-,-,-,-10.98,-14.51,358.73,197.02,2.21,3.07,396.89,204.17,-12.92,-13.56,452.73,336.61,-80.39,-97.53,1367.07,807.67
6343,224.00,253.00,-,-,-,-,-,-,-,-,-9.64,-11.10,336.70,203.32,-1.33,-2.17,350.67,220.05,-24.69,-26.16,444.84,355.03,-81.77,-103.37,1602.08,875.15
6344,227.00,377.00,-,-,-,-,-,-,-,-,-14.42,-19.38,312.38,202.14,-2.73,-6.93,423.30,221.64,-12.71,-18.24,441.00,354.83,-86.40,-47.62,5180.65,966.08
6345,227.00,187.00,-,-,-,-,-,-,-,-,-10.49,-8.92,231.52,194.57,-4.75,-1.97,265.07,232.16,-18.39,-13.52,405.59,367.94,-73.14,-77.79,1329.82,912.10
6346,54.00,4.00,-,-,-,-,-,-,-,-,-2.45,-4.21,518.09,211.47,-2.50,-2.76,687.75,260.82,-27.07,-30.70,755.73,546.97,-86.59,-115.46,2453.82,1211.66
6571,29.00,33.00,-,-,-,-,-,-,-,-,-6.05,-8.10,455.73,205.64,-6.23,-6.36,495.28,257.24,-14.15,-15.94,562.54,376.62,-54.72,-68.81,1878.38,945.73
6572,228.00,342.00,-,-,-,-,-,-,-,-,-10.12,-12.33,270.06,205.81,3.42,5.67,302.26,211.45,-11.32,-11.19,385.52,319.35,-63.21,-76.76,988.64,726.21
6573,746.00,387.00,-,-,-,-,-,-,-,-,-12.04,-12.23,232.07,189.65,-0.01,1.93,212.45,168.63,-25.56,-23.71,334.36,306.91,-51.29,-50.49,637.55,530.26


### wPerp Cmodel Metrics

In [32]:
bundleName = 'wPerpCModel'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,wPerp_cModelFlux_sigmaMAD mmag,wPerp_cModelFlux_median mmag
Tract,,,,
6341,25.00,88.00,-,-
6342,162.00,297.00,-,-
6343,224.00,253.00,-,-
6344,227.00,377.00,-,-
6345,227.00,187.00,-,-
6346,54.00,4.00,-,-
6571,29.00,33.00,-,-
6572,228.00,342.00,-,-
6573,746.00,387.00,-,-


### wPerp PSFP Metrics

In [33]:
# bundleName = 'wPerpPSFP'
# metricTable = metricBundletToDataFrame(bundleName, refs)
# displayTable(tractTable, metricTable)

KeyError: 'wPerpPSFP'

### xPerpCModel Metrics

In [34]:
bundleName = 'xPerpCModel'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,i,r,xPerp_cModelFlux_sigmaMAD mmag,xPerp_cModelFlux_median mmag
Tract,,,,
6341,25.00,88.00,-,-
6342,162.00,297.00,-,-
6343,224.00,253.00,-,-
6344,227.00,377.00,-,-
6345,227.00,187.00,-,-
6346,54.00,4.00,-,-
6571,29.00,33.00,-,-
6572,228.00,342.00,-,-
6573,746.00,387.00,-,-


### xPerpPSFP Metrics

In [35]:
# bundleName = 'xPerpPSFP'
# metricTable = metricBundletToDataFrame(bundleName, refs)
# displayTable(tractTable, metricTable)

KeyError: 'xPerpPSFP'

## Object Table Extended Metrics

In [ ]:
#refs = list(registry.queryDatasets('objectTableExtended_metrics'))

### Ap12PSFSky Metrics

In [25]:
#bundleName = 'ap12PsfSky'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)